In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 📘 Real vs. Nominal GDP Over Time

When tracking economic output over time, it's crucial to differentiate between **Nominal GDP** (measured in current prices) and **Real GDP** (measured in constant base-year prices). Nominal GDP reflects changes in both the quantity of goods produced and their prices, while Real GDP isolates changes in quantity only.

This simulation demonstrates how inflation causes Nominal GDP to grow faster than Real GDP, illustrating the importance of adjusting for price changes to understand true economic growth.

# 🔗 The Relationship: Growth Rates & Deflator

1.  **Nominal GDP Growth ($g_N$):** The percentage change in the current-dollar value of output.
2.  **Real GDP Growth ($g_R$):** The percentage change in the constant-dollar value of output (quantity changes).
3.  **Inflation Rate ($\pi$):** The percentage change in the overall price level.

These three are related by the following approximation (especially accurate for small growth rates):

$$ g_N \approx g_R + \pi $$
Or, more precisely:
$$ (1 + g_N) = (1 + g_R) (1 + \pi) $$

**GDP Deflator:** A price index that measures the overall level of prices of all goods and services included in GDP relative to a base year.
$$ \text{GDP Deflator}_t = \frac{\text{Nominal GDP}_t}{\text{Real GDP}_t} \times 100 $$
The inflation rate ($\pi$) is the percentage change in the GDP Deflator:
$$ \pi_t \approx \frac{\text{Deflator}_t - \text{Deflator}_{t-1}}{\text{Deflator}_{t-1}} $$

In this simulation:
* We specify the constant `real_growth` rate ($g_R$) and the constant `inflation` rate ($\pi$).
* We calculate the path of Real GDP: $Y_{Real, t} = Y_0 (1 + g_R)^t$.
* We calculate the path of the GDP Deflator (assuming base year=100): $P_t = 100 (1 + \pi)^t$.
* We calculate Nominal GDP using the relationship: $Y_{Nominal, t} = \frac{Y_{Real, t} \times P_t}{100}$.

The plot shows the diverging paths of Nominal and Real GDP and the rising GDP Deflator reflecting the assumed inflation.

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Layout
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def plot_real_vs_nominal_gdp(
    base_gdp=1000.0,
    inflation_rate=0.03,
    real_growth_rate=0.02,
    T=20
    ):
    """
    Simulates and plots Nominal GDP, Real GDP, and the GDP Deflator over time.

    Args:
        base_gdp (float): Initial Real GDP (and Nominal GDP) in the base year (t=0).
        inflation_rate (float): Constant annual inflation rate (pi).
        real_growth_rate (float): Constant annual real GDP growth rate (gR).
        T (int): Time Horizon in years.
    """
    # Ensure T is an integer
    T = int(T)
    if T < 1: T = 1

    # Time array (0 to T)
    years_idx = np.arange(T + 1)
    start_year = 2024 # Example start year
    years_actual = start_year + years_idx

    # --- Calculate Paths ---
    # Real GDP Path: Y_Real(t) = Y_Base * (1 + gR)^t
    real_gdp = base_gdp * (1 + real_growth_rate)**years_idx

    # GDP Deflator Path: P(t) = P_Base * (1 + pi)^t (Assume P_Base = 100)
    gdp_deflator = 100.0 * (1 + inflation_rate)**years_idx

    # Nominal GDP Path: Y_Nominal(t) = Y_Real(t) * P(t) / 100
    nominal_gdp = real_gdp * gdp_deflator / 100.0

    # Calculate Nominal Growth Rate (approx = gR + inflation)
    nominal_growth_rate = (1 + real_growth_rate) * (1 + inflation_rate) - 1

    # --- Plotting ---
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Plot Nominal and Real GDP on primary axis
    color_nom = 'darkorange'
    color_real = 'forestgreen'
    ax1.plot(years_actual, nominal_gdp, label=f'Nominal GDP (gN ≈ {nominal_growth_rate:.1%})', linewidth=2, color=color_nom, marker='o', markersize=4)
    ax1.plot(years_actual, real_gdp, label=f'Real GDP (gR = {real_growth_rate:.1%})', linewidth=2, color=color_real, marker='s', markersize=4)
    ax1.fill_between(years_actual, real_gdp, nominal_gdp, color='lightgray', alpha=0.6, label='Inflation Effect')
    ax1.set_xlabel('Year')
    ax1.set_ylabel('GDP ($ Billions, Example)', color='black')
    ax1.tick_params(axis='y', labelcolor='black')
    ax1.grid(True, linestyle='--', alpha=0.7)
    ax1.set_ylim(bottom=0)
    # Format y-axis
    ax1.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:,.0f}'))


    # Create secondary axis for GDP Deflator
    ax2 = ax1.twinx()
    color_defl = 'crimson'
    ax2.plot(years_actual, gdp_deflator, label=f'GDP Deflator (π = {inflation_rate:.1%})', linewidth=2, color=color_defl, linestyle='--', marker='^', markersize=4)
    ax2.set_ylabel(f'GDP Deflator (Base Year = {start_year}, Index=100)', color=color_defl)
    ax2.tick_params(axis='y', labelcolor=color_defl)
    ax2.set_ylim(bottom=0)

    # Combine legends
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

    ax1.set_title('Real vs. Nominal GDP and GDP Deflator Over Time')
    fig.tight_layout()
    plt.show()

    # --- Display Summary ---
    summary_md = f"""
    ### 📊 Growth Summary:

    * **Assumed Real GDP Growth (gR):** {real_growth_rate:.2%} per year
    * **Assumed Inflation Rate (π):** {inflation_rate:.2%} per year
    * **Resulting Nominal GDP Growth (gN ≈ gR + π):** {nominal_growth_rate:.2%} per year

    **Values at Year {years_actual[-1]} (T={T}):**
    * **Nominal GDP:** ${nominal_gdp[-1]:,.2f}$
    * **Real GDP:** ${real_gdp[-1]:,.2f}$
    * **GDP Deflator:** {gdp_deflator[-1]:.1f} (Base Year {start_year}=100)
    """
    display(Markdown(summary_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'}
layout = Layout(width='95%')

interact(
    plot_real_vs_nominal_gdp,
    base_gdp=FloatSlider(value=1000.0, min=100, max=10000, step=100, description='Base Year GDP ($B):', style=style, layout=layout, readout_format=',.0f'),
    inflation_rate=FloatSlider(value=0.03, min=-0.02, max=0.10, step=0.005, description='Inflation Rate (π):', style=style, layout=layout, readout_format='.1%'),
    real_growth_rate=FloatSlider(value=0.02, min=-0.02, max=0.08, step=0.005, description='Real Growth Rate (gR):', style=style, layout=layout, readout_format='.1%'),
    T=IntSlider(value=20, min=5, max=50, step=1, description='Years (T):', style=style, layout=layout)
);


interactive(children=(FloatSlider(value=1000.0, description='Base Year GDP ($B):', layout=Layout(width='95%'),…

# 📘 Real vs. Nominal GDP Over Time

GDP can be measured in **current prices** (nominal) or **constant prices** (real).  
Real GDP tells us what’s happening to quantities — **not prices**.

# 🧠 Nominal vs. Real

| Nominal GDP | Real GDP |
|-------------|----------|
| Current prices | Constant prices |
| Includes price changes | Strips out inflation |
| Grows faster when prices rise | Shows real output growth |

We convert nominal to real using a **GDP deflator**:
\[
\text{Real GDP} = \frac{\text{Nominal GDP}}{\text{Deflator}} \times 100
\]

# 📈 Why This Matters

Imagine GDP goes up 10% — but prices rose 10%. Did real output increase?  
**No!** All the increase is just inflation. Real GDP stays flat.

> Real GDP allows us to **compare across time**, not just see dollar values.
